# Importing the libraries

In [1]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
#from skimage import io
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Spotify API credentials
client_id = 'd4eec2244fb2416ebce8ec488b1f9587'
client_secret = '0cf3c26eca6244e796a10a847dc11f17'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.client.Spotify(auth_manager=auth_manager)

# Importing the dataset

In [3]:
df=pd.read_csv('1M_unique_processed_data.csv')

In [4]:
df.shape

(286205, 31)

In [5]:
df.drop_duplicates(inplace=True)
df.shape

(281000, 31)

In [6]:
# List of columns to drop
columns_to_drop = ['name', 'num_holdouts', 'pid', 'num_tracks', 'num_samples', 'pos', 'time_signature']

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)


In [7]:
# Dropping rows with missing values in specified columns
columns_to_keep = ['artist_name', 'track_name', 'artist_pop', 'popularity']
df = df.dropna(subset=columns_to_keep)

# Selecting only the columns of interest
df = df[columns_to_keep]


In [8]:
df.columns

Index(['artist_name', 'track_name', 'artist_pop', 'popularity'], dtype='object')

In [10]:
df = df.sample(frac=0.1, random_state=42)  # Adjust fraction size as needed

# Display the shape of the sampled dataset
print(df.shape)

(26410, 4)


In [20]:
#df.shape

(264103, 4)

In [22]:
'''
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV

# Assuming 'artist_pop' and 'popularity' are the features we'll use
features = ['artist_name', 'popularity']
X = df[features]

# Define the parameter grid
param_grid = {
    'n_clusters': [5, 8, 10, 12, 15],  # Values to explore for the number of clusters
    'init': ['k-means++', 'random'],  # Different initialization strategies
    'max_iter': [300, 500, 800]  # Maximum number of iterations for convergence
}

# Initialize KMeans model
kmeans = KMeans(random_state=42)

# Create GridSearchCV
# Create GridSearchCV with adjusted_rand_score
grid_search = GridSearchCV(estimator=kmeans, param_grid=param_grid, cv=5, scoring='adjusted_rand_score')

# Fit the GridSearchCV to the data
grid_search.fit(X)

# Get the best parameters and best silhouette score
best_params = grid_search.best_params_
best_score = grid_search.best_score_


print("Best Parameters:", best_params)
print("Best Silhouette Score:", best_score)

'''

TypeError: unhashable type: 'numpy.ndarray'

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
import numpy as np

# One-hot encode the 'artist_name' column
encoder = OneHotEncoder(sparse=False)
artist_encoded = encoder.fit_transform(df[['artist_name']])

# Concatenate the one-hot encoded array with the 'popularity' column
popularity_column = df[['popularity']].values
X = np.concatenate([artist_encoded, popularity_column], axis=1)

# Define the parameter grid
param_grid = {
    'n_clusters': [5, 8, 10, 12, 15],  # Values to explore for the number of clusters
    'init': ['k-means++', 'random'],  # Different initialization strategies
    'max_iter': [300, 500, 800]  # Maximum number of iterations for convergence
}

# Initialize KMeans model
kmeans = KMeans(random_state=42)

# Create GridSearchCV
grid_search = GridSearchCV(estimator=kmeans, param_grid=param_grid, cv=5, scoring='adjusted_rand_score')

# Fit the GridSearchCV to the data
grid_search.fit(X)

# Get the best parameters and best silhouette score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Silhouette Score:", best_score)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/mode

Best Parameters: {'init': 'k-means++', 'max_iter': 300, 'n_clusters': 5}
Best Silhouette Score: nan


In [12]:
# Get the best parameters
best_params = grid_search.best_params_

# Initialize KMeans with the best parameters
best_kmeans = KMeans(n_clusters=best_params['n_clusters'], 
                     init=best_params['init'], 
                     max_iter=best_params['max_iter'], 
                     random_state=42)

# Fit the best KMeans model to the data
best_kmeans.fit(X)

# Assign clusters to the data using the best model
df['cluster'] = best_kmeans.labels_


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [36]:
# Extract track ID from Spotify URL
spotify_url = "https://open.spotify.com/track/2aPTvyE09vUCRwVvj0I8WK?si=7b06b8f1012041ce"
track_id = spotify_url.split('/')[-1].split('?')[0]


In [37]:
# Get the track information
track_info = sp.track(track_id)

# Extract track popularity
track_popularity = track_info['popularity']

# Get the artist information
artist_id = track_info['artists'][0]['id']
artist_info = sp.artist(artist_id)

# Extract artist popularity
artist_popularity = artist_info['popularity']


In [39]:
# Concatenate the selected features of the given track
track_features = np.concatenate([encoder.transform([[artist_info['name']]]), [[track_popularity]]], axis=1)

# Find the cluster of the given track based on selected features
track_cluster = best_kmeans.predict(track_features)[0]

# Select similar tracks from the same cluster
num_recommendations = 5  # Adjust the number of recommendations
similar_tracks = df[df['cluster'] == track_cluster].sample(num_recommendations)

# Show the recommended tracks
recommended_tracks = similar_tracks[['artist_name', 'track_name']]
print(recommended_tracks)

        artist_name            track_name
125908   The Weeknd      I Feel It Coming
130095  Blackstreet            No Diggity
38340   Spice Girls  Wannabe - Radio Edit
179726     DJ Snake    Turn Down for What
259179     Mountain     Mississippi Queen


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [40]:
def get_recommendations(track_url):
    # Extract the track ID from the URL
    track_id = track_url.split('/')[-1].split('?')[0]

    # Get the track information
    track_info = sp.track(track_id)

    # Extract the artist name
    artist_name = track_info['artists'][0]['name']

    # Filter the dataset for tracks by the extracted artist
    artist_tracks = df[df['artist_name'] == artist_name]

    # Get recommendations based on the artist's tracks
    recommendations = artist_tracks.sample(3)  # Adjust the number as needed

    return recommendations[['artist_name', 'track_name']]

# Example usage
spotify_track_url = 'https://open.spotify.com/track/2aPTvyE09vUCRwVvj0I8WK?si=7b06b8f1012041ce'
recommendations = get_recommendations(spotify_track_url)
print(recommendations)

      artist_name          track_name
29681  A$AP Rocky            Multiply
29648  A$AP Rocky            Everyday
29824  A$AP Rocky  Wild for the Night


In [28]:
'''
# Find the cluster of the given track based on 'artist_pop' and 'popularity'
track_cluster = best_kmeans.predict([[artist_popularity, track_popularity]])[0]

# Select similar tracks from the same cluster
similar_tracks = df[df['cluster'] == track_cluster].sample(3)  # Change '3' to get more or fewer recommendations

# Show the recommended tracks
similar_tracks[['artist_name', 'track_name']]
'''


"\n# Find the cluster of the given track based on 'artist_pop' and 'popularity'\ntrack_cluster = best_kmeans.predict([[artist_popularity, track_popularity]])[0]\n\n# Select similar tracks from the same cluster\nsimilar_tracks = df[df['cluster'] == track_cluster].sample(3)  # Change '3' to get more or fewer recommendations\n\n# Show the recommended tracks\nsimilar_tracks[['artist_name', 'track_name']]\n"

In [ ]:
'''
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm

def get_track_id(sp, artist_name, track_name):
    results = sp.search(q=f'artist:{artist_name} track:{track_name}', type='track', limit=1)
    
    if results['tracks']['items']:
        return results['tracks']['items'][0]['id']
    else:
        return None

def get_recommendations(sp, track_id):
    recommendations = sp.recommendations(seed_tracks=[track_id], limit=5)
    return recommendations['tracks']

# Replace 'YOUR_CLIENT_ID' and 'YOUR_CLIENT_SECRET' with your Spotify API credentials
client_id = 'd4eec2244fb2416ebce8ec488b1f9587'
client_secret = '0cf3c26eca6244e796a10a847dc11f17'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

current_artist_name = "Lana Del Rey"
current_track_name = "Young And Beautiful"

track_id = get_track_id(sp, current_artist_name, current_track_name)

if track_id:
    recommended_tracks = get_recommendations(sp, track_id)

    print("Recommended Tracks:")
    for track in tqdm(recommended_tracks, desc="Fetching Recommendations", unit="track"):
        print(f"Track Name: {track['name']}")
        print(f"Artist: {track['artists'][0]['name']}")
        print(f"Track URL: {track['external_urls']['spotify']}")
        print(f"Artist URL: {track['artists'][0]['external_urls']['spotify']}")
        print(f"Album URL: {track['album']['external_urls']['spotify']}")
        print(f"Album Name: {track['album']['name']}")
        print("\n")

else:
    print("Track not found.")
'''

'\nimport spotipy\nfrom spotipy.oauth2 import SpotifyClientCredentials\nfrom tqdm import tqdm\n\ndef get_track_id(sp, artist_name, track_name):\n    results = sp.search(q=f\'artist:{artist_name} track:{track_name}\', type=\'track\', limit=1)\n    \n    if results[\'tracks\'][\'items\']:\n        return results[\'tracks\'][\'items\'][0][\'id\']\n    else:\n        return None\n\ndef get_recommendations(sp, track_id):\n    recommendations = sp.recommendations(seed_tracks=[track_id], limit=5)\n    return recommendations[\'tracks\']\n\n# Replace \'YOUR_CLIENT_ID\' and \'YOUR_CLIENT_SECRET\' with your Spotify API credentials\nclient_id = \'d4eec2244fb2416ebce8ec488b1f9587\'\nclient_secret = \'0cf3c26eca6244e796a10a847dc11f17\'\n\nclient_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)\nsp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)\n\ncurrent_artist_name = "Lana Del Rey"\ncurrent_track_name = "Young And Beautiful"\